COGS 4290 RSA

David Halpern 4/4/23

# How could we know if my mind represents concepts similarly to yours?

If I see a square and you see a square, do our internal representations look the same? [Shepard & Chipman (1970)](https://www.sciencedirect.com/science/article/abs/pii/0010028570900022) note, following arguments from Wittgenstein, that there is basically no way to know. "If there is, as we suppose, some internal event that corresponds to our perception of a square, our ability to form an association between this event and the word “square” requires only that this event have a regular relation to the external object is one of causality, not of structural isomorphism. Such an event could be the activation of some group of neurons, such, perhaps, as a “cell assembly” of the sort described by Hebb (1949). To insist, in addition, that these neurons must be spatially arranged in precisely the form of a square, themselves, does not in the least help to explain how they come to trigger the naming response “square.” On the contrary, it only attempts the absurdity of putting off until later the whole process of pattern recognition that must by definition preceed the pivotal event in question. (With about as much logic, one might as well argue that the neurons that signal that the square is green should themselves be green!)"

So what can we know about the nature of your representations and my representations? Way before the advent of modern cognitive neuroscience, Shepard & Chipman tried to tackle this problem in the following way: they gave participants a deck of 105 cards with the names of two states on them. They asked the participants to sort the deck from most similar to least similar. They then did the same thing again but with pictures of the states instead of names. Using these ranks, they used a statistical procedure called multidimensional scaling to try to map out what the state representations must have looked like in order to generate this data. This is what they got:

<center>
    <img src="figures/ChipShep70_fig1.png" width=400>
</center>

Amazingly, the results looked pretty similar whether they were ranking by image or by name! And they could give a (rough) interpretation of the similarity space:

<center>
    <img src="figures/ChipShep70_fig2.png" width=400>
</center>

Most importantly, subjects were pretty consistent across not only with themselves across the image vs. name ranking but also with the group!

<center>
    <img src="figures/ChipShep70_tab1.png" width=400>
</center>

The key insight of Shepard and Chipman was to look at the "second order isomorphism" -- it's difficult to know if my representation of Louisiana is similar to yours because we can't just ask. But if both of us think Louisiana is more similar to Florida than Colorado, that gets us a lot of the way there. Of course, with modern cognitive neuroscience methods, we can now ask a new question: does my neural activity represent things similarly to yours?

# How should we measure the similarity of neural activity?

With people, we just asked which of these are more similar. But with groups of neurons or electrodes or other sources of neural measurement (e.g. fMRI voxels), there are lots of ways to talk about similarity. For instance, consider A, B, C in figure A below:

<center>
    <img src="figures/geom_distance.png" width=400>
</center>

Imagine the points A, B and C are the responses of two electrodes to seeing three names of states. How similar are each are these? One natural way to think about similarity is Euclidean distance. This is the actual distance in the plane represented by the two electrode responses in figure A. How far apart are A and C? On Measure 1, A is C and C is 2 so they are a distance "2" away. On measure two, A is 0 and C is 2 again so the same distance. If we draw a straight line between A and C, (if you remember from high school geometry class), it would have a length $\sqrt{2^2 + 2^2} = 4$. B is the same distance from A because it is a distance of 2 along Measure 1 (but in the opposite direction as C) and 2 along Measure 2. B and C are the same on Measure 2 but 4 away on Measure 1 so a line between them would have length $\sqrt{4^2 + 0^2} = 4$. So all three points are the same distance away in terms of Euclidean distance. Therefore, we might say, from the neural activity's perspective, all three states are equally similar.

However, imagine we sampled a lot of data from the two neural measures and it looks like the black dots in figure B above. The two measures tend to be correlated. In some sense, this might imply that B should be less similar to A than C because it would be rare for the measures to take on those values. A sligthly more complicated distance measure, known as Mahalanobis distance, takes into account these statistical dependencies when computing distance. There is more detail about how it's calculated in your book.

A separate concern that often comes up with neural measures is that sometimes the numbers aren't meaningful in specific ways. For instance, if two people have slightly different skull thickness then the voltage picked up by scalp EEG will vary in magnitude. If the magnitudes of the points in the figure above doubled, their Euclidean distances would be multipled by \sqrt{2}. This is not necessarily ideal for comparing across subjects and other types of similar artifacts can make it hard to compare signals across sessions from the same subject or even within session. Therefore, it is often preferable to measure similarity using angle-based measures. These types of measures, which include cosine similarity and correlation, look at the angle between the two vectors represented by two points relative to the 0 point. The cosine similarity between a point at \[1, 2\] and \[10, 20\] is 1 because these points have the exact same angle relative to the origin. The figure below, from [Walther et al. (2016)](https://www.sciencedirect.com/science/article/pii/S1053811915011258), shows how various changes to the signals affect distance-based and angle-based distance measures.

<center>
    <img src="figures/WaltEtal16_fig1_w_caption.png" width=400>
</center>

Walther et al. given an argument for why they prefer Euclidean/Mahalanobis distances to angle-based distances when investigating neural activity that is very selective for certain kinds of stimuli (e.g. FFA for faces/PPA for places) and discuss cross-validation of distances measures to reduce bias. However, these concerns are somewhat more relevant for perception studies than memory studies for reasons that are beyond the scope of this class. Because most of the historical literature has used correlation/angle-based measures, we will use that for the examples in this class.

In the 01_compute_rsa notebook, we compute the RSA matrices for several subjects in the categorized free recall dataset. In 01_analyze_rsa notebook, we try to replicate various results in the literature.

In [1]:
import numpy as np
import xarray as xr

Generate a dataset where we observe 10 neural features across 15 states

In [2]:
# mean_features = np.random.normal(size=10)

In [3]:
A = np.diag(np.ones(15))  # a diagonal covariance matrix

In [4]:
A

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.

Add in a couple true correlations across "states"

In [5]:
A[1, 2] = A[2, 1] = .5
A[6, 8] = A[6, 8] = .8
A

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0.5, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0.8, 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ,
       

In [6]:
features = np.random.multivariate_normal(size=10, mean=np.zeros(15), cov=A).T

/tmp/ipykernel_19359/3651426059.py:1: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  features = np.random.multivariate_normal(size=10, mean=np.zeros(15), cov=A).T


Now we want to look at two sets of features and see how similar they are

In [7]:
features[1], features[2]

(array([-0.43379346,  1.11056149, -0.49299213,  0.15266431, -1.68110034,
        -0.31972948, -1.94607026, -1.0665946 , -0.54219678, -1.81629959]),
 array([ 1.00343752, -0.69745606,  0.25493622, -0.27750345, -1.11521085,
        -1.89805719, -0.96418741, -1.61375958, -0.83055134, -2.23989175]))

Generally feature 1 is negative when feature 2 is negative and positive when feature 2 is positive so they seem pretty similar (this is because we made them similar when we generated the covariance matrix). This should be reflected when we compute their correlation. [Pearson's correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is the covariance of the two variables divided by the standard deviation of each multiplied together.

Covariance is the expectation (aka expected value aka mean) of (feature 1 - mean(feature 1)) * (feature 2 - mean(feature 2). This measure is going to be high if feature 1 is usually above its mean when feature 2 is also above its mean. Lets compute it

In [8]:
np.mean(features[1]), np.mean(features[2])

(-0.7035550830662693, -0.8378243885510441)

In [9]:
(features[1] - np.mean(features[1])) 

array([ 0.26976162,  1.81411658,  0.21056296,  0.8562194 , -0.97754525,
        0.3838256 , -1.24251518, -0.36303952,  0.16135831, -1.1127445 ])

In [10]:
(features[1] - np.mean(features[1])) * (features[2] - np.mean(features[2]))

array([ 4.96701804e-01,  2.54644514e-01,  2.30094904e-01,  4.79757655e-01,
        2.71157819e-01, -4.06944492e-01,  1.57007975e-01,  2.81695139e-01,
        1.17356745e-03,  1.56014275e+00])

Now we take the average of that value (but its a sample statistic so we use n-1 in the denominator, instead of n=10)

In [11]:
feat_cov = (np.sum((features[1] - np.mean(features[1])) * 
        (features[2] - np.mean(features[2]))) / (10 - 1)) 
feat_cov 

0.36949240425857705

We can check it against `numpy`'s version

In [12]:
np.cov(features[1], features[2])

array([[0.90933711, 0.3694924 ],
       [0.3694924 , 0.96699552]])

Now just divide that by the sample standard deviations

In [13]:
feat_cov / (np.std(features[1], ddof=1) * np.std(features[2], ddof=1))

0.39403139482584754

And check that against numpy's versiosn of correlation

In [14]:
np.corrcoef(features[1], features[2])

array([[1.        , 0.39403139],
       [0.39403139, 1.        ]])

Now we want a way to compute these correlations for all of the states and keep track of all of the state info. Like we do with EEG, we'll store it using an xarray DataArray

In [15]:
states_da = xr.DataArray(data=features, 
             dims=["states", "features"], 
             coords={'features': ['feat_' + str(i) for i in range(10)],
                     'states': ['Minn.', 'Ore.', 'W.V.', 'Colo.', 'Ala.', 'Ill.',
                               'Nev.', 'Nebr.', 'Okla.', 'Ida.', 'Fla.', 'La.', 
                                'S.C.', 'Mo.', 'Me.']})

In [16]:
states_da

<xarray.DataArray (states: 15, features: 10)>
array([[ 1.37215638e+00, -7.83107444e-01,  8.92483932e-01,
         5.63187438e-01, -1.16171661e+00, -1.14882936e+00,
        -1.06963550e+00,  4.03290538e-01, -1.62234016e-01,
         1.36070817e+00],
       [-4.33793459e-01,  1.11056149e+00, -4.92992128e-01,
         1.52664313e-01, -1.68110034e+00, -3.19729483e-01,
        -1.94607026e+00, -1.06659460e+00, -5.42196776e-01,
        -1.81629959e+00],
       [ 1.00343752e+00, -6.97456059e-01,  2.54936225e-01,
        -2.77503450e-01, -1.11521085e+00, -1.89805719e+00,
        -9.64187412e-01, -1.61375958e+00, -8.30551336e-01,
        -2.23989175e+00],
       [ 5.94887636e-01, -1.24865655e+00,  5.60149872e-01,
        -9.96767906e-01, -2.40219445e-01,  1.49301084e-01,
        -1.27478842e+00, -4.01429527e-02,  6.60049652e-01,
        -1.36063546e+00],
       [ 3.59557537e-01, -4.31148603e-01,  7.35953002e-01,
        -2.14882999e+00,  1.17020396e+00,  8.78985497e-01,
         7.16996352e-02, -8.63983045e-01, -9.61085361e-01,
         1.15166003e-01],
...
       [-6.83531032e-01, -7.35254668e-01, -6.75114441e-01,
        -9.52789130e-01, -2.96145459e-01, -3.24211692e-01,
        -5.13694373e-01, -5.72521129e-01, -1.40842485e+00,
        -1.25655834e+00],
       [-2.72107712e-01, -1.31984045e+00, -9.17576541e-01,
         1.01976647e+00, -9.62932806e-01,  1.02183561e+00,
        -6.64378481e-01, -1.36606490e-02, -5.55235027e-01,
         1.30264145e-01],
       [-6.48281955e-01,  8.79237594e-01,  2.35956556e+00,
        -2.24019186e-01,  7.63582161e-01,  2.54336915e-01,
        -6.92728358e-01, -2.64595536e-01, -1.42657933e-02,
         8.43803081e-01],
       [-1.54264204e-01,  8.77165916e-01,  5.77952425e-01,
         2.78119151e-01, -7.90523265e-01,  8.31894247e-04,
        -2.65530468e-02,  1.67754486e+00,  1.74306821e-01,
        -1.84118532e+00],
       [-5.63427054e-01,  2.01428325e-01,  8.16498163e-01,
        -2.16605475e-01,  1.32603713e+00, -3.44098747e-01,
         6.16866642e-01,  5.42792552e-01,  4.25102602e-01,
         1.56930949e-01]])
Coordinates:
  * features  (features) <U6 'feat_0' 'feat_1' 'feat_2' ... 'feat_8' 'feat_9'
  * states    (states) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'

Now we can use the xr.corr function to compute the correlation of features between states but its not so straightforward because the states have the same names in both -- just plugging it in like this leads to the wrong answer because it matches the array along its dimensions. This leads to just correlates each state with itself, which is going to  always be 1

In [17]:
xr.corr(states_da, states_da, dim='features')

<xarray.DataArray (states: 15)>
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Coordinates:
  * states   (states) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'

What we want is each state correlated with every other one. We need to rename the states variable to get the right answer

In [18]:
states_da2 = states_da.rename({'states': 'states2'})

In [19]:
states_corr = xr.corr(states_da, states_da2, dim='features')

In [20]:
states_corr

<xarray.DataArray (states: 15, states2: 15)>
array([[ 1.        , -0.00492726,  0.34842749,  0.17174179, -0.23337099,
        -0.13591034,  0.69448876,  0.18135156,  0.21698459,  0.29641959,
        -0.5161642 ,  0.18917061,  0.10813791, -0.13309291, -0.349114  ],
       [-0.00492726,  1.        ,  0.39403139,  0.0942525 , -0.37306048,
         0.25842193, -0.02125713,  0.33918399, -0.70006337, -0.08637994,
        -0.09322863,  0.02056622,  0.13790546,  0.51686023, -0.46754347],
       [ 0.34842749,  0.39403139,  1.        ,  0.40686224, -0.06283689,
         0.11093024,  0.20453605,  0.21297332,  0.31743528, -0.08524908,
         0.01635774, -0.28530581,  0.00133324,  0.25510926, -0.18887923],
       [ 0.17174179,  0.0942525 ,  0.40686224,  1.        ,  0.26056696,
        -0.4182726 ,  0.46762417,  0.27303398,  0.04699816,  0.48556959,
         0.07005099, -0.02782859,  0.10066491,  0.26752461, -0.01817243],
       [-0.23337099, -0.37306048, -0.06283689,  0.26056696,  1.        ,
        -0.25913125, -0.1687046 , -0.10595822,  0.3088522 ,  0.094556  ,
         0.55672834, -0.3266098 ,  0.40073681, -0.38220462,  0.29355157],
       [-0.13591034,  0.25842193,  0.11093024, -0.4182726 , -0.25913125,
         1.        , -0.40524194,  0.19756771, -0.18702301, -0.44873954,
        -0.3515078 , -0.631269  ,  0.40628731,  0.00151319,  0.46693423],
       [ 0.69448876, -0.02125713,  0.20453605,  0.46762417, -0.1687046 ,
        -0.40524194,  1.        ,  0.36759451,  0.19048984,  0.24744178,
...
        -0.18702301,  0.19048984, -0.23186602,  1.        , -0.06854393,
         0.27557108, -0.1872446 , -0.21595844, -0.30444075,  0.31182934],
       [ 0.29641959, -0.08637994, -0.08524908,  0.48556959,  0.094556  ,
        -0.44873954,  0.24744178,  0.18927904, -0.06854393,  1.        ,
        -0.33618377,  0.21864981, -0.32129133, -0.28730831, -0.3984487 ],
       [-0.5161642 , -0.09322863,  0.01635774,  0.07005099,  0.55672834,
        -0.3515078 , -0.03285235, -0.15040685,  0.27557108, -0.33618377,
         1.        , -0.05807488, -0.00277709,  0.235368  ,  0.20294309],
       [ 0.18917061,  0.02056622, -0.28530581, -0.02782859, -0.3266098 ,
        -0.631269  ,  0.22264917, -0.55697988, -0.1872446 ,  0.21864981,
        -0.05807488,  1.        , -0.36716175, -0.1095829 , -0.62713105],
       [ 0.10813791,  0.13790546,  0.00133324,  0.10066491,  0.40073681,
         0.40628731,  0.03217684,  0.07581911, -0.21595844, -0.32129133,
        -0.00277709, -0.36716175,  1.        , -0.0848183 ,  0.42762532],
       [-0.13309291,  0.51686023,  0.25510926,  0.26752461, -0.38220462,
         0.00151319,  0.37780766,  0.59329296, -0.30444075, -0.28730831,
         0.235368  , -0.1095829 , -0.0848183 ,  1.        ,  0.01584123],
       [-0.349114  , -0.46754347, -0.18887923, -0.01817243,  0.29355157,
         0.46693423, -0.10044317,  0.06975515,  0.31182934, -0.3984487 ,
         0.20294309, -0.62713105,  0.42762532,  0.01584123,  1.        ]])
Coordinates:
  * states   (states) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'
  * states2  (states2) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'

In [21]:
states_corr_df = states_corr.to_dataframe('corr').reset_index()
states_corr_df

,states,states2,corr
0,Minn.,Minn.,1.000000
1,Minn.,Ore.,-0.004927
2,Minn.,W.V.,0.348427
3,Minn.,Colo.,0.171742
4,Minn.,Ala.,-0.233371
...,...,...,...
220,Me.,Fla.,0.202943
221,Me.,La.,-0.627131
222,Me.,S.C.,0.427625
223,Me.,Mo.,0.015841


This is the same as we calculated before!

In [22]:
states_corr_df.query('states == "Ore." and states2 == "W.V."')

,states,states2,corr
17,Ore.,W.V.,0.394031
